# i-Eye Saffron Image Classifier

In [26]:
from keras.models import load_model
import ipywidgets as widgets
import io
from PIL import Image
import numpy as np
from keras.preprocessing.image import img_to_array
from IPython import display
import whatimage
import pyheif

In [27]:
model = load_model('vgg_model_1.hdf5')
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_5 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               3211392   
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 2)                

In [28]:
for idx in range(len(model.layers)):
    print(model.layers[idx].get_config())

{'name': 'vgg16', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 224, 224, 3), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_3'}, 'name': 'input_3', 'inbound_nodes': []}, {'class_name': 'Conv2D', 'config': {'name': 'block1_conv1', 'trainable': False, 'dtype': 'float32', 'filters': 64, 'kernel_size': (3, 3), 'strides': (1, 1), 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'groups': 1, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}, 'name': 'block1_conv1', 'inbound_nodes': [[['input_3', 0, 0, {}]]]}, {'class_name': 'Conv2D', 'config': {'name': 'block1_conv2', 'trainable': False, 'dtype': 'float32', 'filters': 64, 'kernel_size': (3, 3), 'str

In [29]:
def show_my_image():
    for name, file_info in btn_upload.value.items():
        img = decodeImage((file_info['content'])) 
    #return the image 
    return img.resize((400,400))

In [30]:
def decodeImage(bytesIo):
    fmt = whatimage.identify_image(bytesIo)
    if fmt in ['heic','heif']:
        i = pyheif.read_heif(bytesIo)
        pi = Image.frombytes(mode=i.mode, size=i.size, data=i.data)
    else:
        pi = Image.open(io.BytesIO(bytesIo))
    return pi

In [31]:
def predict(image: Image.Image):
    for name, file_info in btn_upload.value.items():
        img = decodeImage((file_info['content']))
    img = img_to_array(img.resize(size=(224,224)))
    img = np.expand_dims(img, axis = 0)
    img = img / 127.5 - 1.0    
    result = model.predict(img)    
    resp = {}  
    lbl_pred.value = "Predicted Class : " + "Saffron" if np.argmax(result)==1 else "Non-Saffron"
    saffron_prob.value = "Probability of Saffron " + f"{result[0][1]*100:0.2f}%"  
    nsaffron_prob.value = "Probability of Non-Saffron " + f"{result[0][0]*100:0.2f}%"
    # clear the output
    out_pl.clear_output()
     # display the uploaded image
    with out_pl:
        display.display(show_my_image())

In [32]:
# Elements Of the App
# upload Button
btn_upload = widgets.FileUpload()
# output widget to display image 
out_pl = widgets.Output()
# label for the result of prediction
lbl_pred = widgets.Label()
saffron_prob = widgets.Label()
nsaffron_prob = widgets.Label()
# Button to classify the uploaded image 
btn_run = widgets.Button(description='Classify')
# event for classify button to predict the result and display uploaded image
btn_run.on_click(predict)

In [33]:
widgets.VBox([widgets.Label('Select Image'),btn_upload,btn_run,out_pl,lbl_pred,saffron_prob,nsaffron_prob],box_style='info')

VBox(box_style='info', children=(Label(value='Select Image'), FileUpload(value={}, description='Upload'), Butt…